In [ ]:
'''
We want to define a quality species entry by having all three of the bacterial markers which
would indicate if they have a full genomic sequence completed and is not just an species entry with a one off protein entry. 
So I am going to take the full bacterial dataset with the CXC filtering then Ill get the numbers breaking down the
number of each oxidase. This notebook also adds the human readable names of the phylum for figures.
'''

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import Bio
import numpy as np
import os 


In [ ]:
 
fbds = pd.read_excel(r"C:\PATH\4_Filtered_FBDS_IPRcounts_byspecies_MAUA.xlsx")


In [ ]:
print(fbds.shape,'\n',fbds.columns,'\n',fbds)

In [ ]:
# big three iprs IPR047873', 'IPR010243', 'IPR036286
species = fbds['species'].count()

species_filter_3 = fbds[(fbds['IPR047873'] != 0) & 
                        (fbds['IPR010243'] != 0) & 
                        (fbds['IPR036286'] != 0)]['species'].count()

species_filter_2_RNA_SP = fbds[(fbds['IPR047873'] == 0) & 
                        (fbds['IPR010243'] != 0) & 
                        (fbds['IPR036286'] != 0)]['species'].count()

species_filter_2_RB_SP = fbds[(fbds['IPR047873'] != 0) & 
                        (fbds['IPR010243'] == 0) & 
                        (fbds['IPR036286'] != 0)]['species'].count()

species_filter_2_RB_RNA = fbds[(fbds['IPR047873'] > 0) & 
                        (fbds['IPR010243'] != 0) & 
                        (fbds['IPR036286'] == 0)]['species'].count()

species_filter_0 = fbds[(fbds['IPR047873'] == 0) & 
                        (fbds['IPR010243'] == 0) & 
                        (fbds['IPR036286'] == 0)]['species'].count()

species_filter_rb = fbds[(fbds['IPR047873'] != 0) & 
                        (fbds['IPR010243'] == 0) & 
                        (fbds['IPR036286'] == 0)]['species'].count()

species_filter_rna = fbds[(fbds['IPR047873'] == 0) & 
                        (fbds['IPR010243'] != 0) & 
                        (fbds['IPR036286'] == 0)]['species'].count()

species_filter_sp = fbds[(fbds['IPR047873'] == 0) & 
                        (fbds['IPR010243'] == 0) & 
                        (fbds['IPR036286'] != 0)]['species'].count()

In [ ]:
#these numbers drop when I look at phylum that have 10 or more species and are not candidate
print('total species: ', species)
print('bacterial species with all three quality markers: ', species_filter_3)
print('bacterial species with RNA polymerase and Signal peptidase: ', species_filter_2_RNA_SP)
print('bacterial species with ribosomal marker and Signal peptidase: ', species_filter_2_RB_SP)
print('bacterial species with RNA polymerase and ribosomal marker: ', species_filter_2_RB_RNA)
print('bacterial species with only ribosomal marker: ', species_filter_rb)
print('bacterial species with only RNA polymerase: ', species_filter_rna)
print('bacterial species with only signal peptidase: ', species_filter_sp)
print('bacterial species with none: ', species_filter_0)

In [ ]:

species_filter_3 = (fbds['IPR047873'] != 0) & (fbds['IPR010243'] != 0) & (fbds['IPR036286'] != 0)


fbds_2 = fbds.loc[species_filter_3]
print(fbds.shape,'\n',fbds_2.shape)


In [ ]:
# now im copying over the tax id function i wrote earlier so  i gotta import more shit
#this is the import section
#I need to trim this I think some are uneeded
from Bio import SeqIO
from io import StringIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import sys, errno, re, json, ssl
from urllib import request
from urllib.error import HTTPError
from time import sleep
import os
import re
import csv
from Bio import Entrez

In [ ]:
tax_LOT = pd.read_pickle(r"C:\PATH\ncbi_2025_taxonomy_table.pkl")

In [ ]:
email = 'example@your_email.com' 



print(tax_LOT)
email = 'example@your_email.com' 


def fetch_TAXID_data(tax_id, email):
    classification = tax_id
    Entrez.email = email  
    retry_limit = 3  

    attempts = 0
    while attempts < retry_limit:
        try:
            handle = Entrez.efetch(db="taxonomy", id=str(classification), retmode="xml")
            records = Entrez.read(handle)
            for rec in records:
                NAME = rec.get("ScientificName")
            break  
        except Exception as e:
            print(f"HTTP error: {str(e)} - Sleeping 10s and retrying for ID {classification}")
            sleep(2)
            attempts += 1  
        finally:
            if 'handle' in locals() and handle:
                handle.close()  
    if attempts == retry_limit:
        print(f"Failed after {retry_limit} retries for ID {classification}")

    return NAME

def fetch_LINEAGE_NAME_data(lineage_list, email):
    Entrez.email = email
    lineage_df = {}
    
    for classification in lineage_list: # same as before
        try:
            NAME_series = tax_LOT[tax_LOT['tax_id'] == classification]['name_txt']
            if not NAME_series.empty:
                NAME = NAME_series.iloc[0]  
                if NAME:
                    lineage_df[NAME] = classification
                    print(f'NAME: {NAME}, Tax ID: {classification}')
                else:
                    print(f'No rank available for Tax ID: {classification}')
            else:
                print(f'No entry found for Tax ID: {classification}, fetching from NCBI...')
                NAME = fetch_TAXID_data(classification, email)
                if NAME and NAME != "No NAME available":
                    lineage_df[NAME] = classification
                else:
                    print(f'Failed to fetch data for Tax ID: {classification} from NCBI.')
        except Exception as e:
            print(f'Error processing Tax ID {classification}: {str(e)}')

    return lineage_df

# testing everything still works (0 fails):
list_tax = ['1', '0', '2624677', '1644055', '2157', '1236']
email = 'your_email@example.com'  # Replace with your actual email
print(fetch_LINEAGE_NAME_data(list_tax, email))

In [ ]:
fbds_2 = fbds_2.fillna(0)
phy_list = [ int(x) for x in fbds_2['phylum']]

In [ ]:
phy_list_2 = list(set(phy_list))
print(phy_list_2)

In [ ]:
lineages = fetch_LINEAGE_NAME_data(phy_list_2,email)

In [ ]:
print(lineages)

In [ ]:
print(fbds_2)

In [ ]:
fbds_2['scientific_names_phylum'] = ''


In [ ]:
print(fbds_2)

In [ ]:
phy_list = [ int(x) for x in fbds_2['phylum']]

In [ ]:
for i in fbds_2.index:
    i_phy = fbds_2.at[i,'phylum']
    i_phy_int=int(i_phy)
    if i_phy_int == 0:
        sci_name = 'no_rank'
        fbds_2.at[i,'scientific_names_phylum'] = sci_name
    else:
        sci_name = list(lineages.keys())[list(lineages.values()).index(i_phy_int)]
        fbds_2.at[i,'scientific_names_phylum'] = sci_name

In [ ]:
print(fbds_2['scientific_names_phylum'])

In [ ]:
#after the quality markers are checked the species level data moves on to the analysis here
fbds_2.to_excel(r"C:\PATH\5_Filtered_FBDS_IPRcounts_byspecies_MAUA_names.xlsx")